In [ ]:
class test_1(object):
    def __init__(self):
        self.model_path = self.create_model_path()
        x_train, y_train, x_valid, y_valid = Standardization(origin_data)
        self.X_train = x_train
        self.Y_train = y_train
        self.X_valid = x_valid
        self.Y_valid = y_valid
        self.shear_range = 0.2
        self.zoom_range = 0.2
        self.width_shift_range=0.05,
        self.height_shift_range=0.05,
        self.Image_generator = ImageDataGenerator(shear_range = self.shear_range, zoom_range = self.zoom_range, width_shift_range = self.width_shift_range, height_shift_range = self.height_shift_range)
        self.checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath = self.model_path+'best_model.h5', monitor='val_accuracy', save_best_only=True)
        self.early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience= 10)

    def create_model_path(self):
        model_path=Root_dir + 'test_1/'
        if not os.path.exists(model_path):
            os.mkdir(model_path)
        return model_path

    def create_module(self, layer, channel_size, kernel_size):
        conv_layer = tf.keras.layers.Conv2D(channel_size, kernel_size= kernel_size, kernel_initializer = tf.keras.initializers.Orthogonal(), strides = 1, padding = 'same', activation = 'relu')(layer)
        batch_layer = tf.keras.layers.BatchNormalization()(conv_layer)
        conv_layer = tf.keras.layers.Conv2D(channel_size, kernel_size= kernel_size, kernel_initializer = tf.keras.initializers.Orthogonal(), strides = 1, padding = 'same', activation = 'relu')(batch_layer)
        batch_layer = tf.keras.layers.BatchNormalization()(conv_layer)
        pooling_layer = tf.keras.layers.MaxPooling2D((2, 2))(batch_layer)
        return pooling_layer

    def create_model(self):
        inputs = tf.keras.layers.Input(shape = (self.X_train.shape[1:]))
        module_1 = self.create_module(inputs, 64, 2)
        module_2 = self.create_module(module_1, 64, 2)
        flatten_layer = tf.keras.layers.Flatten()(module_2)
        outputs=tf.keras.layers.Dense(10, activation='softmax')(flatten_layer)
        return tf.keras.models.Model(inputs=inputs, outputs=outputs)

    def define_model(self):
        self.model = self.create_model()
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    def learn(self):
        self.define_model()
        training_generator = self.Image_generator.flow(self.X_train, self.Y_train, batch_size=32, shuffle=True)
        validation_generator = self.Image_generator.flow(self.X_valid, self.Y_valid, batch_size=32, shuffle=True)
        self.model.fit_generator(training_generator,epochs=150, callbacks = [self.checkpoint, self.early_stopping],
                shuffle=True,
                validation_data = validation_generator,

                )

In [ ]:
test_1 = test_1()
test_1.learn()

In [ ]:
def Standardization(origin_data):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, validation_index in split.split(origin_data, origin_data["digit"]):
        strat_train_set = origin_data.iloc[train_index]
        strat_validation_set = origin_data.iloc[validation_index]
    X_train = strat_train_set.iloc[:,2:].values.reshape(-1,28,28,1)
    X_valid = strat_validation_set.iloc[:,2:].values.reshape(-1,28,28,1)
    X_train = (X_train - np.mean(X_train))/255.0
    X_valid = (X_valid - np.mean(X_valid))/255.0
    Y_train = strat_train_set['digit']
    Y_valid = strat_validation_set['digit']
    return X_train, Y_train, X_valid, Y_valid

In [ ]:
def Normalize(origin_data):
    X = origin_data.iloc[:,2:].values.reshape(-1,28,28,1)
    X = X/255.0
    Y = origin_data['digit']

    return X, Y